In [ ]:
import xarray as xr

xr.set_options(display_expand_data=False)

In [ ]:
from erlab.interactive.exampledata import generate_data

dat = generate_data()

In [ ]:
dat

In [ ]:
dat.sel(ky=0.3, method="nearest")

How about the Fermi surface?

In [ ]:
dat.sel(eV=0.0, method="nearest")

In [ ]:
dat.sel(eV=slice(-0.025, 0.025)).mean("eV")

In [ ]:
dat.qsel(eV=0.0, eV_width=0.05)

In [ ]:
dat.qsel(kx=slice(-0.3, 0.3), ky=0.3, eV=0.0, eV_width=0.05)